## → Chapter Six 

**Deciphering the broken pinyin, Completing the Epic Merge**  

🔺One of the challanges in data cleaning was that, global databases like ISBNdb and isbnsearch.org often returned Chinese book metadata in **pinyin**(the phonetic romanization of Chinese) and not in readable Chinese characters. To fix this, I used a Python library called **pinyin2hanzi** which converts pinyin text into the most likely Chinese characters. 

🔺With the code below, I was able to transform some of the titles in pinyin into Chinese and clean up the title values from dangdang.com best as I could.

🔺Since there are also English titles mixed in, I specified the confidence score to be > 0.01 to select only the accurate results 


In [17]:
import pandas as pd
from Pinyin2Hanzi import DefaultDagParams, dag

# initialize pinyin converter
dagparams = DefaultDagParams()

# helper function to convert pinyin string to hanzi if score > 0.1
def convert_pinyin_to_hanzi(pinyin_string):
    pinyin_list = pinyin_string.strip().lower().split()
    try:
        result = dag(dagparams, pinyin_list, path_num=5)
        for item in result:
            if item.score > 0.01:
                return ''.join(item.path)
        return None
    except:
        return None

# --- Convert isbndbResults.csv ---
isbndb_df = pd.read_csv('../data/isbndbResults.csv')
isbndb_df['converted_title'] = isbndb_df['title'].astype(str).apply(convert_pinyin_to_hanzi)

# --- Convert zoteroExport.csv ---
zotero_df = pd.read_csv('../data/zoteroExport.csv')

# process title
zotero_df['converted_title'] = zotero_df['Title'].astype(str).apply(convert_pinyin_to_hanzi)

# clean ISBN column (keep first 17 characters, remove hyphens)
zotero_df['ISBN'] = zotero_df['ISBN'].astype(str).str[:17].str.replace('-', '', regex=False)

# result DataFrames
isbndbConverted = isbndb_df
zoteroConverted = zotero_df

In [23]:
import pandas as pd
import re

# Read the CSV
dangdang = pd.read_csv('../data/dangdangResults.csv')

# Clean title: remove all full-width parentheses （） and 【】
def clean_title(title):
    if pd.isna(title):
        return title
    title = re.sub(r'（.*?）', '', title)
    title = re.sub(r'【.*?】', '', title)
    return title.strip()

# Apply cleaning
dangdang['title'] = dangdang['title'].astype(str).apply(clean_title)


🔺However, a lot of these works' titles are too complicated for the library to convert, so I decided to do a manual edit on the final result to: transform the rest of the pinyin and clean the rest of the dangdang titles. 

🔺Then, I created a script to merge the book data collected from the four sources into a single, structured dataset formatted to match the fields required for Shopify product import.
- **ISBNdb** 
- **Zotero** 
- **Dangdang.com** 

In [25]:
import pandas as pd

# Load data
df = pd.read_csv('../data/shopifyTemplate.csv')
dangdang = pd.read_csv('../data/dangdangResults.csv')
zotero = pd.read_csv('../data/zoteroExport.csv')
isbndb = pd.read_csv('../data/isbndbResults.csv')

# Clean up ISBNs to match format
zotero['ISBN'] = zotero['ISBN'].astype(str).str.replace('-', '').str[:13]
isbndb['isbn_searched'] = isbndb['isbn_searched'].astype(str).str[:13]
dangdang['isbn'] = dangdang['isbn'].astype(str).str[:13]

# Helper function to get first non-null value from priority list
def get_priority_value(isbn, zot_col, isbndb_col, dang_col):
    if isbn in zotero['ISBN'].values:
        return zotero.loc[zotero['ISBN'] == isbn, zot_col].values[0]
    elif isbn in isbndb['isbn_searched'].values:
        return isbndb.loc[isbndb['isbn_searched'] == isbn, isbndb_col].values[0]
    elif isbn in dangdang['isbn'].values:
        return dangdang.loc[dangdang['isbn'] == isbn, dang_col].values[0]
    else:
        return None

# Apply field population based on hierarchy
df['Variant Barcode'] = df['Variant Barcode'].astype(str).str[:13]  # Ensure consistency
df['Title'] = df['Variant Barcode'].apply(lambda x: get_priority_value(x, 'Title', 'title', 'title'))
df['Image Src'] = df['Variant Barcode'].apply(lambda x: isbndb.loc[isbndb['isbn_searched'] == x, 'image'].values[0]
                                              if x in isbndb['isbn_searched'].values else None)

# Construct Body (HTML)
def build_html_body(isbn):
    if isbn in zotero['ISBN'].values:
        row = zotero.loc[zotero['ISBN'] == isbn].iloc[0]
        year = row.get('Publication Year', '')
        author = row.get('Author', '')
        publisher = row.get('Publisher', '')
        pages = row.get('Num Pages', '')
        description = row.get('Abstract Note', '')
    elif isbn in isbndb['isbn_searched'].values:
        row = isbndb.loc[isbndb['isbn_searched'] == isbn].iloc[0]
        year = row.get('date_published', '')
        author = row.get('authors', '')
        publisher = row.get('publisher', '')
        pages = row.get('pages', '')
        description = row.get('synopsis', '')
    else:
        return ''

    return f"<p>Year: {year}</p><p>Author: {author}</p><p>Publisher: {publisher}</p><p>Pages: {pages}</p><p>Description: {description}</p>"

df['Body (HTML)'] = df['Variant Barcode'].apply(build_html_body)
df.to_csv('finalInventory.csv', index=False, encoding='utf-8-sig')


due to restrictions, I cannot provide the final data sheet of the full inventory result. The finalInventory.csv file in the data folder is a sample of the final result.